In [1]:
import pandas as pd
import os
import re

In [3]:
# list all the pdfs
pdf_path = os.listdir('SDG-indicator-pdfs')

#add in directory
pdf_path = ['SDG-indicator-pdfs/' + path for path in pdf_path]

# there's one called .ds-store, dont know why, so i deleted it'
#pdf_path.remove('SDG-indicator-pdfs/.DS_Store')


In [8]:
len(pdf_path)

246

In [15]:
#install package
#pip install PyMuPDF

In [7]:
import fitz

def get_text(filepath: str) -> str:
    with fitz.open(filepath) as doc:
        text = ""
        for page in doc:
            text += page.getText()
        return text


ModuleNotFoundError: No module named 'frontend'

In [177]:
import pandas as pd 
lst1 = pdf_path
lst2 = [get_text(path) for path in pdf_path]

percentile_list = pd.DataFrame(
    {'pdf_name': lst1,
     'pdf_text': lst2
    })
percentile_list = percentile_list.sort_values('pdf_name')
percentile_list.to_csv('text_chunk.csv')

In [2]:
text_chunk = pd.read_csv('text_chunk.csv')
text_chunk

,Unnamed: 0,pdf_name,pdf_text
0,217,SDG-indicator-pdfs/Metadata-01-01-01a.pdf,Last updated: July 2021 \nSDG indicator metada...
1,193,SDG-indicator-pdfs/Metadata-01-02-01.pdf,Last updated: August 2021 \nSDG indicator meta...
2,205,SDG-indicator-pdfs/Metadata-01-02-02.pdf,Last updated: May 2021 \nSDG indicator metadat...
3,114,SDG-indicator-pdfs/Metadata-01-03-01a.pdf,Last updated: January 2021 \nSDG indicator met...
4,140,SDG-indicator-pdfs/Metadata-01-04-01.pdf,Last updated: March 2021 \nSDG indicator metad...
...,...,...,...
241,115,SDG-indicator-pdfs/Metadata-17-17-01.pdf,Last updated: September 2020 \nGoal 17: Streng...
242,55,SDG-indicator-pdfs/Metadata-17-18-02.pdf,Last updated: 13 February 2018 \n \nGoal 17: S...
243,50,SDG-indicator-pdfs/Metadata-17-18-03.pdf,Last updated: 11 July 2017 \n \nGoal 17: Stren...
244,165,SDG-indicator-pdfs/Metadata-17-19-01.pdf,Last updated: 11 July 2017 \n \nGoal 17: Stren...


In [34]:
# find the definition from .pdf files

definition_all = {}
d_pattern = 'Definition:'
c_pattern = 'Concepts:'
error = []
for i in range(text_chunk.shape[0]):
    try:
        test_str = text_chunk.iloc[i]['pdf_text']
        definition_index = [(m.start(0), m.end(0)) for m in re.finditer(d_pattern, test_str)]
        concept_goal = [(m.start(0), m.end(0)) for m in re.finditer(c_pattern, test_str)]
        definition = test_str[definition_index[0][1]+2:concept_goal[0][0]]
        definition_all[text_chunk.iloc[i]['pdf_name']] = definition
    except:
        error.append(text_chunk.iloc[i]['pdf_name'])
        #print(text_chunk.iloc[i]['pdf_name'])
        definition_all[text_chunk.iloc[i]['pdf_name']] = ''

text_chunk['definition']= text_chunk['pdf_name'].map(definition_all)
df = pd.DataFrame(definition_all.items())

In [35]:
#keep XX-XX-XXa eg 01-01-01
df['Index'] = df[0].apply(lambda x: x.replace('SDG-indicator-pdfs/Metadata-',"").replace('.pdf',''))
df = df.rename(columns={0:'pdf_name',1:'definition' })

In [36]:
#change 01 02 to 1 2 eg 01-01-01 to 1-01-01
df['Index'] = df['Index'].apply(lambda x: x[1:] if x[0] == '0' else x)
#lowercase the string
df['Index'] = df["Index"]. str. lower() 
#replace -0 with .
df['Index'] = df['Index'].apply(lambda x: x.replace('-0',".").replace('-','.'))
#get rid of the end 'a' eg 1.1.1a to 1.1.1
df['Index'] = df['Index'].apply(lambda x: x.replace('a','') if x[-1] == 'a' else x)

In [147]:
goal_target_indicator = pd.read_excel('Goal_Target_Indicator.xlsx')
# two NaN in subset 
goal_target_indicator = goal_target_indicator.dropna(subset = ['Index'])
len(goal_target_indicator['Index'])

247

In [148]:
goal_target_indicator['Index']

0        1.1.1
1        1.2.1
2        1.2.2
3        1.3.1
4        1.4.1
5        1.4.2
6        1.5.1
7        1.5.2
8        1.5.3
9        1.5.4
10       1.a.1
11       1.a.2
12       1.b.1
13       2.1.1
14       2.1.2
15       2.2.1
16       2.2.2
17       2.2.3
18       2.3.1
19       2.3.2
20       2.4.1
21       2.5.1
22       2.5.2
23       2.a.1
24       2.a.2
25       2.b.1
26       2.c.1
27       3.1.1
28       3.1.2
29       3.2.1
30       3.2.2
31       3.3.1
32       3.3.2
33       3.3.3
34       3.3.4
35       3.3.5
36       3.4.1
37       3.4.2
38       3.5.1
39       3.5.2
40       3.6.1
41       3.7.1
42       3.7.2
43       3.8.1
44       3.8.2
45       3.9.1
46       3.9.2
47       3.9.3
48       3.a.1
49       3.b.1
50       3.b.2
51       3.b.3
52       3.c.1
53       3.d.1
54       3.d.2
55       4.1.1
56       4.1.2
57       4.2.1
58       4.2.2
59       4.3.1
60       4.4.1
61       4.5.1
62       4.6.1
63       4.7.1
64       4.a.1
65       4.b.1
66       4

In [150]:
B = goal_target_indicator['Index']
B

0        1.1.1
1        1.2.1
2        1.2.2
3        1.3.1
4        1.4.1
5        1.4.2
6        1.5.1
7        1.5.2
8        1.5.3
9        1.5.4
10       1.a.1
11       1.a.2
12       1.b.1
13       2.1.1
14       2.1.2
15       2.2.1
16       2.2.2
17       2.2.3
18       2.3.1
19       2.3.2
20       2.4.1
21       2.5.1
22       2.5.2
23       2.a.1
24       2.a.2
25       2.b.1
26       2.c.1
27       3.1.1
28       3.1.2
29       3.2.1
30       3.2.2
31       3.3.1
32       3.3.2
33       3.3.3
34       3.3.4
35       3.3.5
36       3.4.1
37       3.4.2
38       3.5.1
39       3.5.2
40       3.6.1
41       3.7.1
42       3.7.2
43       3.8.1
44       3.8.2
45       3.9.1
46       3.9.2
47       3.9.3
48       3.a.1
49       3.b.1
50       3.b.2
51       3.b.3
52       3.c.1
53       3.d.1
54       3.d.2
55       4.1.1
56       4.1.2
57       4.2.1
58       4.2.2
59       4.3.1
60       4.4.1
61       4.5.1
62       4.6.1
63       4.7.1
64       4.a.1
65       4.b.1
66       4

In [151]:
A = df['Index']
len(A)

246

In [152]:
A

0        1.1.1
1        1.2.1
2        1.2.2
3        1.3.1
4        1.4.1
5        1.4.2
6        1.5.1
7        1.5.2
8        1.5.3
9        1.5.4
10       1.a.1
11       1.a.2
12       1.b.1
13       2.1.1
14       2.1.2
15       2.2.1
16       2.2.2
17       2.2.3
18       2.3.1
19       2.3.2
20       2.4.1
21       2.5.1
22       2.5.2
23       2.a.1
24       2.a.2
25       2.b.1
26       2.c.1
27       3.1.1
28       3.1.2
29       3.2.1
30       3.2.2
31       3.3.1
32       3.3.2
33       3.3.3
34       3.3.4
35       3.3.5
36       3.4.1
37       3.4.2
38       3.5.1
39       3.5.2
40       3.6.1
41       3.7.1
42       3.7.2
43       3.8.1
44       3.8.2
45       3.9.1
46       3.9.2
47       3.9.3
48       3.b.2
49       3.b.3
50       3.c.1
51       3.a.1
52       3.b.1
53       3.d.1
54       3.d.2
55       4.1.1
56       4.1.2
57       4.2.1
58       4.2.2
59       4.3.1
60       4.4.1
61       4.5.1
62       4.6.1
63       4.7.1
64       4.b.1
65       4.c.1
66       4

In [153]:
set(B) - set(A)

{'16.a.1 ',
 '16.b.1 ',
 '17.1.1 ',
 '17.1.2 ',
 '17.18.1',
 '17.2.1 ',
 '17.3.1 ',
 '17.3.2 ',
 '17.4.1 ',
 '17.5.1 ',
 '17.6.1 ',
 '17.7.1 ',
 '17.8.1 ',
 '17.9.1 ',
 '8.10.'}